# Projektni zadatak 4, Genomska informatika, Skolska 2021/2022

# Aleksandar Malovic 2021/3375

## Assignment

* Implement an algorithm for indexed string search using Burrows-Wheeler transform and FM index as described on the lesson slides, without additional optimizations
* Write tests for intermediate functions as well as final algorithm
* Optimize algorithm in regards to memory usage and performace. Run regression tests and check optimization results using assigned testing parameters

## Testing remarks

Tests for individual functions are grouped within testing functions to separate and enclose testing scopes. This also simplifies calling the tests at other places in the code.

## Helper functions

#### Is input a valid string

In [1]:
def isInputValid(t):
    return t != None and len(t) > 0

def testInputValidation():
    # Test case 1: None, should return false
    assert not isInputValid(None)
    
    # Test case 2: Empty string, should return false
    assert not isInputValid('')
    
    # Test case 3: Valid input
    assert isInputValid('abc')

In [2]:
testInputValidation()

#### Is input a valid string with ending character

In [3]:
def isBWInputValid(t):
    return t != None and len(t) > 0 and t != "$" and t.endswith('$')

def testInputValidation():
    # Test case 1: None, should return false
    assert not isBWInputValid(None)
    
    # Test case 2: Empty string, should return false
    assert not isBWInputValid('')
    
    # Test case 3: String containing only the ending character, should return false
    assert not isBWInputValid('$')
    
    # Test case 4: String missing ending character, should return false
    assert not isBWInputValid('abc')
    
    # Test case 5: Valid string, should return true
    assert isBWInputValid('abc$')

In [4]:
testInputValidation()

#### Arays are equal check, both length and order

In [5]:
def arraysEqual(output, expectedOutput):
    # Both inputs being None is considered an error as well
    if output == None or expectedOutput == None:
        return False
    if  len(output) != len(expectedOutput):
        return False
    for i in range(0, len(expectedOutput)):
         if output[i] != expectedOutput[i]:
            return False
    return True

def testArraysEqual():
    assert not arraysEqual(None, None)
    
    assert not arraysEqual(None, [])
    
    assert not arraysEqual(['0', '1', '2'], ['0', '1', '2', '3'])
    
    assert not arraysEqual(['0', '1'], ['2', '3'])
    
    assert arraysEqual(['0', '1', '2', '3'], ['0', '1', '2', '3'])

In [6]:
testArraysEqual()

#### Sets are equal check, order not important, same elements

In [7]:
def setsEqual(output, expectedOutput):
    # Both input being None is considered an error as well
    if output == None or expectedOutput == None:
        return False
    if  len(output) != len(expectedOutput):
        return False
    for e in output:
         if e not in expectedOutput:
            return False
    return True

def testSetsEqual():
    assert not setsEqual(None, None)
    
    assert not setsEqual(None, [])
    
    assert not setsEqual(['0', '1', '2'], ['0', '1', '2', '3'])
    
    assert not setsEqual(['1', '0'], ['2', '3'])
    
    assert setsEqual(['0', '1', '3', '2'], ['2', '0', '1', '3'])

In [8]:
testSetsEqual()

## Burrows-Wheeler transform

Burrows-Wheeler transform consists of three steps.
* Create an array of all input string rotations
* Sort array in alphabetical order (Burrows-Wheeler matrix)
* Take last column of the Burrows-Wheeler matrix

**ASSUMPTION 1:** Input string will already have the ending character appended before being subjected to the BWT, otherwise functions will return None to indicate error. Algorithm could create a local copy with the ending character appended but in case of large strings creating a local copy with just one additional character would be suboptimal from a memory standpoint considering other functions in the FM index rely on the same string.

### Create array of all string rotations

Function appends the string to itself to make it simpler to calculate rotations (based on lesson slides). Implementation using splicing is also possible but is suboptimal from a memory standpoint.

In [9]:
def rotations(t):
    if not isBWInputValid(t):
        return None
    tt = t * 2
    return [ tt[i:i+len(t)] for i in range(0, len(t)) ]

#### Tests

In [10]:
def testRotations():
    # Test case 1: None, should return None
    assert rotations(None) == None
    
    # Test case 2: Empty string, should return false
    assert rotations('') == None
    
    # Test case 3: String containing only the ending character, should return false
    assert rotations('$') == None
    
    # Test case 4: String missing ending character, should return false
    assert rotations('abc') == None
    
    # Test case 5: Input string of just one character 
    inputValue = 'a$'
    expectedOutput = ['a$', '$a']
    output = rotations(inputValue)
    assert arraysEqual(output, expectedOutput)
    
    # Test case 6: Valid input string
    inputValue = 'abcd$'
    expectedOutput = ['abcd$', 'bcd$a', 'cd$ab', 'd$abc', '$abcd']
    output = rotations(inputValue)
    assert arraysEqual(output, expectedOutput)

In [11]:
testRotations()

### Sort string rotations in alphabetical order, Burrows-Wheeler Matrix

Based on lesson slides.

In [12]:
def calculateBurrowsWheelerMatrix(t):
    r = rotations(t)
    return sorted(r) if r != None else None

#### Tests

In [13]:
def testCalculateBurrowsWheelerMatrix():
    # Test case 1: None, should return None
    assert calculateBurrowsWheelerMatrix(None) == None
    
    # Test case 2: Empty string, should return None
    assert calculateBurrowsWheelerMatrix('') == None
    
    # Test case 3: String containing only the ending character, should return None
    assert calculateBurrowsWheelerMatrix('$') == None
    
    # Test case 4: String missing ending character, should return None
    assert calculateBurrowsWheelerMatrix('abc') == None
    
    # Test case 5
    inputValue = 'abcd$'
    expectedOutput = ['$abcd','abcd$', 'bcd$a', 'cd$ab', 'd$abc']
    output = calculateBurrowsWheelerMatrix(inputValue)
    arraysEqual(output, expectedOutput)

In [14]:
testCalculateBurrowsWheelerMatrix()

### Generate final Burrows-Wheeler transform

We take the last column of the sorted rotations matrix (based on lesson slides)

In [15]:
# Calculates the actual Burrows-Wheeler transform, or L index (last column of the matrix)
def calculateLIndex(t):
    return ''.join(map(lambda x: x[-1], t)) if t != None else None

# Calculates the F index (first column of the matrix), needed for FM index later
def calculateFIndex(t):
    return ''.join(map(lambda x: x[0], t)) if t != None else None

def calculateBurrowsWheelerTransform(t):
    r = calculateBurrowsWheelerMatrix(t)
    return calculateLIndex(r) if r != None else None

#### Tests

In [16]:
def testCalculateLIndex():
    # Test case 1: None, should return None
    assert calculateLIndex(None) == None
    
    # Test case 2:
    assert calculateLIndex(['$abcd','abcd$', 'bcd$a', 'cd$ab', 'd$abc']) == 'd$abc'
    
def testCalculateFIndex():
    # Test case 1: None, should return None
    assert calculateFIndex(None) == None
    
    # Test case 2:
    assert calculateFIndex(['$abcd','abcd$', 'bcd$a', 'cd$ab', 'd$abc']) == '$abcd'

def testBurrowsWheelerTransform():
    # Test case 1: None, should return None
    assert calculateBurrowsWheelerTransform(None) == None
    
    # Test case 2: Empty string, should return None
    assert calculateBurrowsWheelerTransform('') == None
    
    # Test case 3: String containing only the ending character, should return None
    assert calculateBurrowsWheelerTransform('$') == None
    
    # Test case 4: String missing ending character, should return None
    assert calculateBurrowsWheelerTransform('abc') == None
    
    # Test case 5
    inputValue = 'abcd$'
    expectedOutput = 'd$abc'
    output = calculateBurrowsWheelerTransform(inputValue)
    assert output != None
    assert output == expectedOutput
    
    # Test case 6
    inputValue = 'abaaba$'
    expectedOutput = 'abba$aa'
    output = calculateBurrowsWheelerTransform(inputValue)
    assert output != None
    assert output == expectedOutput

In [17]:
testCalculateLIndex()

testCalculateFIndex()

testBurrowsWheelerTransform()

## FM index

Core of the FM index structure consists of the following data:
* F index (first column of the Burrows-Wheeler matrix)
* L index (last column of the Burrows-Wheeler matrix, the Burrows-Wheeler transform itself)
* Tally (matrix of input string character ranks)
* Suffix array of the input string

### Calculate tally

Tally is a matrix of L index(referred to as input string for simplicity in following paragraph) character ranks.
Each row is assigned to one of the characters of the input string (except the terminal character). The number of columns is equal to the length of the input string. The value of a particular field in the matrix is the rank of the particular character at that point of the input string, which represents how many occurences of said character have been in the input string so far.

In [31]:
def calculateTally(t):
    if not isInputValid(t):
        return None
    # Start with empty tally
    tally = {}
    for i in range(0, len(t)):
        # Copy previous column values to current one
        for value in tally.values():
            value[i] = value[i-1]
        """ 
        Take current character in input string.
        If a row for said character exists, increment rank. If not, insert a row populated with 0s, and then increment.
        """
        currentChar = t[i]
        if currentChar not in tally:
            tally[currentChar] = [0] * len(t)
        tally[currentChar][i] += 1
    return tally

#### Tests

In [34]:
def testCalculateTally():
    # Test case 1: None, should return None
    assert calculateTally(None) == None
    
    # Test case 2: Empty string, should return none
    assert calculateTally('') == None
    
    # Test case 3: One character string
    inputValue = 'a'
    expectedOutput = {
        'a': [1]
    }
    output = calculateTally(inputValue)
    assert output != None
    assert setsEqual(output.keys(), expectedOutput.keys())
    for key in output.keys():
        assert arraysEqual(output[key], expectedOutput[key])
    
    # Test case 4: Valid BWT string
    inputValue = 'abcaab$c'
    expectedOutput = {
        '$': [0, 0, 0, 0, 0, 0, 1, 1],
        'a': [1, 1, 1, 2, 3, 3, 3, 3],
        'b': [0, 1, 1, 1, 1, 2, 2, 2],
        'c': [0, 0, 1, 1, 1, 1, 1, 2]
    }
    output = calculateTally(inputValue)
    assert output != None
    assert setsEqual(output.keys(), expectedOutput.keys())
    for key in output.keys():
        assert arraysEqual(output[key], expectedOutput[key])

In [35]:
testCalculateTally()

### Calculate suffix array

Suffix array is a sorted array of all suffixes of an input string. The array itself contains tuples (offset, suffix), sorted by suffix, where offset is the offset of the suffix within the input string. In case of an FM index, we are storing a suffix array of the original input string.

In [21]:
def calculateSuffixArray(t):
    if not isInputValid(t):
        return None
    suffixArray = []
    for i in range(0, len(t)):
        suffixArray.append((i, t[i:]))
    return sorted(suffixArray, key=lambda x: x[1])

#### Tests

In [22]:
def testCalculateSuffixArray():
    # Test case 1: None, should return None
    assert calculateSuffixArray(None) == None
    
    # Test case 2: Empty string, should return none
    assert calculateSuffixArray('') == None
    
    # Test case 3: String with one character
    inputValue = 'a'
    expectedOutput = [(0, 'a')]
    output = calculateSuffixArray(inputValue)
    assert arraysEqual(output, expectedOutput)
    
    # Test case 4: Valid string
    inputValue = 'abcaabc$'
    expectedOutput = [(7, '$'),(3,'aabc$'), (4,'abc$'), 
                      (0,'abcaabc$'), (5,'bc$'), (1,'bcaabc$'),(6,'c$'),(2,'caabc$')]
    output = calculateSuffixArray(inputValue)
    assert arraysEqual(output, expectedOutput)

In [23]:
testCalculateSuffixArray()

### FMIndex class with query support

FM index querying is performed in the following way.

If we are looking for P, start with characters in P in reverse order. Find positions of P's shortest suffix, and then extend the suffix until we exhaust P, or are not able to calculate the position of the next suffix, in which case we know there is no match.

**EXAMPLE:**

Let us assume R is ABCD, string we are querying is T. 

* Start with D and calculate first and last position of D based on the F Index.

 **\[start, end\] = \[index of first occurence of D in F index + 1, index of last occurence of D in F index\]**
 
* Now we are looking for CD. Using LF mapping, we can see the rows of L index that contain C before D using the F index and the tally.

 **\[start, end\] = \[
index of first occurence of C in F index + tally(C, start -1) + 1, 
index of first occurence of C in F index + tally(C, end)
\]**

* Continue repeating the process until we exhaust R, or until we get an invalid value.

In [95]:
class FMIndex:
    def __init__(self, t):
        """
        Check if input is a valid string.
        If input doesn't already have the ending character, append the ending character.
        """
        if not isInputValid(t):
            raise ValueError("Invalid input string for calculating FM index")
        tt = t
        if not tt.endswith('$'):
            tt += '$'
        self.t = tt
        self.fIndex = None
        self.lIndex = None
        self.tally = None
        self.suffixArray = None
        
    """
    Calculates the FM index fields based on assigned input string
    """
    def calculate(self):
        bwm = calculateBurrowsWheelerMatrix(self.t)
        self.fIndex = calculateFIndex(bwm)
        self.lIndex = calculateLIndex(bwm)
        self.tally = calculateTally(self.lIndex)
        self.suffixArray = calculateSuffixArray(self.t)
    
    """
    Queries the initial string t for substring p. Returns array of indexes within t where p is located.
    """
    def query(self, p):
        start = 0
        end = 0
        length = len(p)
        for i in range(0, length):
            currentChar = p[length - i - 1]
            if start == 0 and end == 0: # first character, look just in F index
                start = self.fIndex.index(currentChar)
                end = self.fIndex.rindex(currentChar)
            else:
                firstOcc = self.fIndex.index(currentChar)
                start = firstOcc + self.tally[currentChar][start - 1]
                end = firstOcc + self.tally[currentChar][end]
        return [suffix[0] for suffix in self.suffixArray[start:end]]

#### Tests

In [103]:
def _testFMIndex(t, p, expectedOutput, expectedFailure):
    try:
        fmIndex = FMIndex(t)
    except ValueError:
        return True if expectedFailure else False
    fmIndex.calculate()
    output = fmIndex.query(p)
    assert output != None
    assert arraysEqual(output, expectedOutput)
    del fmIndex
    return True

def testFMIndex():
    # Test case 1: None
    assert _testFMIndex(None, '', [], True)
    assert not _testFMIndex(None, '', [], False)
    
    # Test case 2: Empty string
    assert _testFMIndex('', '', [], True)
    assert not _testFMIndex('', '', [], False)
    
    #Test case 3: Simple string without ending character
    assert _testFMIndex('testtest', 'te', [4, 0], False)
    
    #Test case 4: Simple string with ending character
    assert _testFMIndex('testtest$', 'te', [4, 0], False)

In [104]:
testFMIndex()